In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mylib as ml

# 데이터 로드 (예: CSV 파일)
data = ml.dataLoad(660)
data

AttributeError: module 'mylib' has no attribute 'dataLoad'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mylib as ml

# 데이터 로드 (예: CSV 파일)
data = ml.dataLoad()

# 종가, 매도량, 매수량으로부터 입력과 타겟 생성
X = data[['매도량', '매수량']].values
y = (data['종가'].shift(-1) > data['종가']).astype(int).values[:-1]  # 다음 날 종가가 오르면 1, 내리면 0

# 마지막 행 제거
X = X[:-1]

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 및 트랜스포머 모델 정의
def create_combined_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # CNN 부분
    x = tf.keras.layers.Conv1D(64, kernel_size=2, activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(32, kernel_size=2, activation='relu')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    
    # Flatten 후 트랜스포머 입력으로 변환
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=64)(tf.expand_dims(x, axis=1), tf.expand_dims(x, axis=1))
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    # 출력층
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 생성
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # 3D 텐서로 변환
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

combined_model = create_combined_model((X_train_cnn.shape[1], 1))

# 모델 학습
combined_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.2)

# 모델 평가
loss, accuracy = combined_model.evaluate(X_test_cnn, y_test)

print(f"결합 모델 정확도: {accuracy:.4f}, 손실: {loss:.4f}")
